In [43]:
import os, sys
sys.path.insert(1, os.getcwd()+'/..')
if (os.path.basename(os.getcwd()) == 'tutorials'):
    os.chdir('../..')

from src.msa import generate
from src.msa.visualization import plot
from src.msa.feature_extraction import features

from mspc_pca.pca import *
from mspc_pca.omeda import *
from mspc_pca.mspc import *
from mspc_pca.ckf import *
import mspc_pca.plot as pca_plot

import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# %matplotlib inline
import matplotlib.pyplot as plt
plt.close('all')
%matplotlib qt


#### Generate synthetic signal

In [44]:
T = 60 #s. Duration in secods
sr = 100 # Sampling rate
n_samples = int(T*sr)

time = np.linspace(0,T, n_samples)

# Generate Chirps
chirp = np.zeros_like(time)
# chirp  += generate.chirp(time, A=1.0, f0=0.0, f1=50., t0=1. ,t1=60 ,method='linear',phi=0)*generate.decay(time, 0.0, 1.)
# chirp  += generate.chirp(time, A=1.0, f0=50., f1=0.0, t0=1. ,t1=60 ,method='linear',phi=0)*generate.decay(time, 0.0, 1.)
# chirp  += generate.chirp(time, A=4.0, f0=40., f1=10., t0=10 ,t1=25 ,method='linear',phi=0)*generate.decay(time, 0.5, 10)
# chirp  += generate.chirp(time, A=1.5, f0=10., f1=15., t0=25 ,t1=30 ,method='linear',phi=0)*generate.decay(time, 0.1, 25)

chirp  += generate.chirp(time, A=5.0, f0=45., f1=15., t0=5. ,t1=6. ,method='linear',phi=0)*generate.decay(time, 0.5, 5.) # F falls qucikly in time
chirp  += generate.chirp(time, A=6.0, f0=0., f1=30, t0=33 ,t1=33.1 ,method='linear',phi=0)*generate.decay(time, 0.6, 33) # F grows quickly in time

chirp  += generate.chirp(time, A=2.0, f0=25., f1=25.1, t0=15 ,t1=35 ,method='linear',phi=0)*generate.decay(time, 0.1, 15) # F grows slowly in time
chirp  += generate.chirp(time, A=2.0, f0=45., f1=44.9,t0=40 ,t1=60 ,method='linear',phi=0)*generate.decay(time, 0.2, 40) # F falls slowly in time

chirp  += generate.chirp(time, A=2.0, f0=15., f1=20,t0=45 ,t1=50 ,method='linear',phi=0)*generate.decay(time, 0.1, 45) 

# Generate white noise
A = 2
np.random.rand(123)
white = np.random.rand(int(T*sr))*A*2 - A

# Synthetic signal
signal = chirp + white 

#### Spectrograms: Parameters

**samplig rate (sr)**: Sampling rate is often fixed by our sensors. Via Nyquist, it determines the maximum frequency we can observe.

**window size**: Longer windows provide better frequency resolution and worse temporal resolution.

**shift**: Shorter shifts provide better temporal resolution.

**nº of bins**: Number of different frequencies to consider for the FFT. Its value should be equal to the number of samples per window. If lower, the frequency resolution will deminish.


In [45]:
# Parameters

sr = sr #Hz. 
window_sizes = [1, 3, 6] #s. 
shift_fractions = [1.0, 0.5, 0.1]#s. 
n_bins = None # 


scale = False
if scale:
    vmin = 0
    vmax = 100000.0
    norm = vmax-vmin
else: vmin, vmax, norm = None, None, 1

fig, ax = plot.grid(len(window_sizes), len(shift_fractions), text0=fr"Window size", text1=rf"Window shift", figsize=(9,9))

N = len(window_sizes)
M = len(shift_fractions)

times = [[[] for _ in range(M)] for _ in range(N)]
freqs = [[[] for _ in range(M)] for _ in range(N)]
Sxxs  = [[[] for _ in range(M)] for _ in range(N)]


for n, window in enumerate(window_sizes):
    for m, shift_fraction in enumerate(shift_fractions):
            win_samples = int(sr*window)
            hop = int(win_samples*shift_fraction)
            time, freq, Sxx = features.spectrogram(signal, sr, win_samples, hop, "blackman","zeros", "linear")

            # Eliminate padded times
            mask = (time >=0) & (time <=T)
            time = time[mask]
            Sxx = Sxx[:, mask]

            times[n][m] = time
            freqs[n][m] = freq
            Sxxs[n][m] = Sxx

            _, _ , mesh = plot.spectrogram(time, freq, Sxx/norm, (0,T), (0,sr/2), "nipy_spectral", False, vmin, vmax, ax = ax[m,n], logscale=False)


bbox = ax[-1, -1].get_position()
tbox = ax[ 0, -1].get_position()
cax = fig.add_axes([0.92, bbox.y0, 0.02, tbox.y1-bbox.y0])  # [left, bottom, width, height] en coords de figura
cbar = fig.colorbar(mesh, cax=cax)
cbar.ax.set_yticks([])      
cbar.ax.set_yticklabels([])
cbar.set_label("Relative intensity", rotation=270, labelpad=18, fontsize=16)

ax[-1,1].set_xlabel("Time (s)", fontsize=16);
ax[1,0].set_ylabel("Frequency (Hz)", fontsize=16);

#### PCA Visualization

In [46]:
autoscale = False


fig, ax = plot.grid(len(window_sizes), len(shift_fractions), text0=fr"Window size", text1=rf"Window shift", figsize=(9,9), )

for n, window in enumerate(window_sizes):
    for m, shift_fraction in enumerate(shift_fractions):
        Sxx = Sxxs[n][m]
        data = Sxx.T
        scaler = StandardScaler(with_std=autoscale)
        data_norm = scaler.fit_transform(data)
        pca_plot.var_pca(data, 6, autoscale, ax = ax[m,n])
        ax[m, n].set_title('')
        ax[m, n].set_xlabel('')
        ax[m, n].set_ylabel('')

        # Remove legend (if present)
        legend = ax[m, n].get_legend()
        if legend is not None:
            legend.remove()


In [47]:
pca = PCA(n_components=2)

fig, ax = plot.grid(len(window_sizes), len(shift_fractions), text0=fr"Window size", text1=rf"Window shift", figsize=(9,9), )

for n, window in enumerate(window_sizes):
    for m, shift_fraction in enumerate(shift_fractions):
        Sxx = Sxxs[n][m]
        data = Sxx.T
        scaler = StandardScaler(with_std=autoscale)
        data_norm = scaler.fit_transform(data)
        freqs_label = [f"{round(x, 2)}Hz" for x in freqs[n][m]]
        times_label = [f"{round(x, 2)}s" for x in times[n][m]]
        _,_,scatter = pca_plot.biplot(data_norm, pca, 1, 2, label_dist=0.3, score_labels=times_label, loading_labels=freqs_label, score_classes=times[n][m], size=10, ax=ax[m,n])
        ax[m, n].set_title('')
        # ax[m, n].set_xlabel('')
        # ax[m, n].set_ylabel('')

        # Remove legend (if present)
        legend = ax[m, n].get_legend()
        if legend is not None:
            legend.remove()



bbox = ax[-1, -1].get_position()
tbox = ax[ 0, -1].get_position()
cax = fig.add_axes([0.92, bbox.y0, 0.02, tbox.y1-bbox.y0])  # [left, bottom, width, height] en coords de figura
cbar = fig.colorbar(scatter, cax=cax)
# cbar.ax.set_yticks([])      
# cbar.ax.set_yticklabels([])
cbar.set_label("Time (s)", rotation=270, labelpad=18, fontsize=16)

In [48]:
times[n][m]

array([ 0. ,  0.6,  1.2,  1.8,  2.4,  3. ,  3.6,  4.2,  4.8,  5.4,  6. ,
        6.6,  7.2,  7.8,  8.4,  9. ,  9.6, 10.2, 10.8, 11.4, 12. , 12.6,
       13.2, 13.8, 14.4, 15. , 15.6, 16.2, 16.8, 17.4, 18. , 18.6, 19.2,
       19.8, 20.4, 21. , 21.6, 22.2, 22.8, 23.4, 24. , 24.6, 25.2, 25.8,
       26.4, 27. , 27.6, 28.2, 28.8, 29.4, 30. , 30.6, 31.2, 31.8, 32.4,
       33. , 33.6, 34.2, 34.8, 35.4, 36. , 36.6, 37.2, 37.8, 38.4, 39. ,
       39.6, 40.2, 40.8, 41.4, 42. , 42.6, 43.2, 43.8, 44.4, 45. , 45.6,
       46.2, 46.8, 47.4, 48. , 48.6, 49.2, 49.8, 50.4, 51. , 51.6, 52.2,
       52.8, 53.4, 54. , 54.6, 55.2, 55.8, 56.4, 57. , 57.6, 58.2, 58.8,
       59.4, 60. ])